In [2]:
# -*- coding: utf-8 -*-
from pathlib import Path
import re
import unicodedata
import pandas as pd

# === CONFIG ===
INPUT  = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\04_SAIDA_MAXENT\processamento\7_excel")
OUTPUT = Path(r"D:\ARTIGOS\2025\LARANJA_MAX\07_POS_ANALISE\15_PAISES")
SHEET_NAME = "resumo"
OUTPUT.mkdir(parents=True, exist_ok=True)

PAISES_ORDEM = [
    "Brazil","China","India","Mexico","Spain","Egypt","United States",
    "Indonesia","Iran","South Africa","Pakistan","Italy","Algeria",
    "Morocco","Vietnam"
]

# mapeia variações comuns -> nome canônico
ALIASES = {
    # EUA
    "United States of America": "United States",
    "U.S.": "United States",
    "USA": "United States",
    "United States": "United States",
    # Irã
    "Iran (Islamic Republic of)": "Iran",
    "IR Iran": "Iran",
    "Iran": "Iran",
    # Vietnã (caso apareça como duas palavras)
    "Viet Nam": "Vietnam",
}

def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def normaliza_txt(x: object) -> str:
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)  # espaços múltiplos
    return s

def canoniza_pais(nome: str) -> str:
    n = normaliza_txt(nome)
    # aplica alias direto se bater
    if n in ALIASES:
        return ALIASES[n]
    # fallback: tenta remover complemento entre parênteses
    n_sem_par = re.sub(r"\s*\(.*?\)\s*", "", n).strip()
    if n_sem_par in ALIASES:
        return ALIASES[n_sem_par]
    # fallback extra: compara sem acentos/maiusc.
    n_key = strip_accents(n).lower()
    for k, v in ALIASES.items():
        if strip_accents(k).lower() == n_key:
            return v
    return n  # sem mapeamento: retorna como veio

for xlsx in sorted(INPUT.glob("*.xlsx")):
    try:
        df = pd.read_excel(xlsx, sheet_name=SHEET_NAME, engine="openpyxl")
    except ValueError as e:
        print(f"⚠️  {xlsx.name}: aba '{SHEET_NAME}' não encontrada. Pulando. ({e})")
        continue
    except Exception as e:
        print(f"⚠️  {xlsx.name}: erro ao abrir. Pulando. ({e})")
        continue

    if "name" not in df.columns:
        print(f"⚠️  {xlsx.name}: coluna 'name' não existe na aba '{SHEET_NAME}'. Pulando.")
        continue

    # normaliza e canoniza nomes de países
    df["name"] = df["name"].map(canoniza_pais)

    # filtra e ordena (mantém múltiplas linhas por país, ex.: por gridcode)
    df_filtrado = df[df["name"].isin(PAISES_ORDEM)].copy()
    df_filtrado["__ordem_pais__"] = pd.Categorical(
        df_filtrado["name"], categories=PAISES_ORDEM, ordered=True
    )
    sort_cols = ["__ordem_pais__"] + (["gridcode"] if "gridcode" in df_filtrado.columns else [])
    df_filtrado = df_filtrado.sort_values(sort_cols).drop(columns="__ordem_pais__")

    # relatório
    encontrados = df_filtrado["name"].drop_duplicates().tolist()
    faltantes = [p for p in PAISES_ORDEM if p not in encontrados]

    # salva com o mesmo nome
    destino = OUTPUT / xlsx.name
    try:
        with pd.ExcelWriter(destino, engine="openpyxl", mode="w") as writer:
            df_filtrado.to_excel(writer, sheet_name=SHEET_NAME, index=False)
        print(
            f"✅  {xlsx.name}: {len(encontrados)}/15 países, "
            f"{len(df_filtrado)} linhas salvas em '{destino.name}'."
            + (f" Faltantes: {', '.join(faltantes)}." if faltantes else "")
        )
    except Exception as e:
        print(f"❌  {xlsx.name}: erro ao salvar. ({e})")


✅  2021_2040_ssp126.xlsx: 15/15 países, 59 linhas salvas em '2021_2040_ssp126.xlsx'.
✅  2021_2040_ssp245.xlsx: 15/15 países, 15 linhas salvas em '2021_2040_ssp245.xlsx'.
✅  2021_2040_ssp370.xlsx: 15/15 países, 15 linhas salvas em '2021_2040_ssp370.xlsx'.
✅  2021_2040_ssp585.xlsx: 15/15 países, 15 linhas salvas em '2021_2040_ssp585.xlsx'.
✅  2041_2060_ssp126.xlsx: 15/15 países, 15 linhas salvas em '2041_2060_ssp126.xlsx'.
✅  2041_2060_ssp245.xlsx: 15/15 países, 15 linhas salvas em '2041_2060_ssp245.xlsx'.
✅  2041_2060_ssp370.xlsx: 15/15 países, 15 linhas salvas em '2041_2060_ssp370.xlsx'.
✅  2041_2060_ssp585.xlsx: 15/15 países, 15 linhas salvas em '2041_2060_ssp585.xlsx'.
✅  2061_2080_ssp126.xlsx: 15/15 países, 15 linhas salvas em '2061_2080_ssp126.xlsx'.
✅  2061_2080_ssp245.xlsx: 15/15 países, 15 linhas salvas em '2061_2080_ssp245.xlsx'.
✅  2061_2080_ssp370.xlsx: 15/15 países, 15 linhas salvas em '2061_2080_ssp370.xlsx'.
✅  2061_2080_ssp585.xlsx: 15/15 países, 15 linhas salvas em '2061